<a href="https://colab.research.google.com/github/manuel-alvarez/scheduling/blob/master/poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Schedule Generator with Genetic Algorithms

## POC

In [0]:
import numpy

In [0]:
POPULATION_SIZE = 11
INDIVIDUAL_SIZE = 5
RESOURCES = 5 

In [0]:
def initialize_population(population_size, individual_size, resources_size):
  population = None
  for i in range(population_size):
    if population is None:
      population = numpy.array([numpy.random.randint(resources_size, size=individual_size)])
    else:
      population = numpy.append(population, [numpy.random.randint(resources_size, size=individual_size)], axis=0)
  return population

In [0]:
population = initialize_population(POPULATION_SIZE, INDIVIDUAL_SIZE, RESOURCES)